# Usuarios y títulos

In [1]:
import ast
import csv
import itertools
import os
import re
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

## Préstamos

In [2]:
%%time

merge_df = pd.read_csv('merge.csv', header=0)
merge_df = merge_df.fillna('')

CPU times: user 1.48 s, sys: 68.4 ms, total: 1.54 s
Wall time: 1.54 s


In [3]:
merge_df.shape

(516206, 12)

In [4]:
merge_df.head()

,TITULO,AUTOR,DEWEY,FECHA_PUB,PAIS_EJEMPLAR,COD_BARRAS_LECTOR,CODBAR,Sucursal,Tratamiento,Edad,Ciudad,Rango_edad
0,Johannes Gutenberg,"Borràs Perelló, Lluís.",925,2010,Spain,88007424,88007424,ARM,sr.,43,Armenia,40-49
1,Genealogía de una bruja,"Lacombe, Benjamin, 1982-",843,2009,Spain,88007424,88007424,ARM,sr.,43,Armenia,40-49
2,La independencia de Colombia : así fue,"Vasco, Irene, 1952-",986,2009,Colombia,88007424,88007424,ARM,sr.,43,Armenia,40-49
3,Maravillas del mundo,"Steele, Philip.",910,2007,Mexico,88007424,88007424,ARM,sr.,43,Armenia,40-49
4,No te gustaría ser un adivino maya,"Matthews, Rupert.",972,2008,Colombia,88007424,88007424,ARM,sr.,43,Armenia,40-49


## Thresholds

In [5]:
THRESHOLD_GROUPS = 100

In [6]:
THRESHOLD_DEWEY = 50

In [7]:
TOP_GROUP = 10

## Groups

In [8]:
group_count_sr = merge_df.groupby(['Ciudad', 'Tratamiento', 'Rango_edad']).CODBAR.count()

In [9]:
len(group_count_sr)

333

In [10]:
sorted_group_count_sr = group_count_sr.sort_values(ascending=False)

In [11]:
pd.DataFrame(sorted_group_count_sr)

CODBAR
Ciudad       Tratamiento Rango_edad        
Bogotá       sr.         18-29        46986
             sra.        18-29        42716
             sr.         30-39        31544
Tunja        sra.        -17          26232
Bogotá       sra.        30-39        22022
Tunja        sr.         -17          17622
Bogotá       sr.         50-64        16693
                         40-49        16089
Popayán      sra.        -17          12217
             sr.         -17           8630
Pasto        sra.        -17           7970
             sr.         -17           7738
Bogotá       sra.        40-49         7399
Tunja        sra.        18-29         6745
Bogotá       sra.        50-64         6483
Neiva        sr.         18-29         5628
Tunja        sr.         18-29         5503
Pereira      sra.        -17           5347
Pasto        sr.         18-29         5248
Popayán      sr.         18-29         5092
Pasto        sra.        18-29         5081
Bogotá       sr.         +65           5003
Tunja        sra.        30-39         4809
Popayán      sra.        18-29         4729
Ipiales      sra.        -17           4277
             sr.         -17           3834
Neiva        sra.        18-29         3405
Pereira      sr.         18-29         3352
                         -17           3247
Pasto        sra.        30-39         3188
...                                     ...
Quibdó       sr.         40-49           20
Manizales    sra.        +65             16
Valledupar   sra.        50-64           16
Quibdó       sra.        40-49           14
Montería     sr.         -17             13
Ibagué       sra.        +65             13
Cúcuta       sr.         18-29           12
Cali         sra.        40-49           12
Cúcuta       sra.        40-49           11
Buenaventura sra.        50-64            9
Barranquilla sra.        +65              9
Manizales    sra.        -17              9
Medellín     sra.        -17              8
Popayán      sra.        +65              8
Cali         sra.        50-64            7
Cúcuta       sra.        +65              6
                         30-39            6
Valledupar   sra.        +65              6
Cúcuta       sra.        50-64            5
Medellín     sr.         -17              4
Florencia    sra.        +65              4
San Andrés   sra.        50-64            3
Leticia      sra.        40-49            3
Montería     sra.        -17              3
Quibdó       sra.        50-64            3
Girardot     sra.        +65              3
Buenaventura sra.        +65              3
Leticia      sra.        +65              2
Sincelejo    sra.        +65              2
Montería     sra.        50-64            1

[333 rows x 1 columns]

In [12]:
filtered_group_count_sr = group_count_sr[lambda x: x >= THRESHOLD_GROUPS]

In [13]:
len(filtered_group_count_sr)

245

In [14]:
sorted_filtered_group_count_sr = filtered_group_count_sr.sort_values(ascending=False)

In [15]:
pd.DataFrame(sorted_filtered_group_count_sr.head())

CODBAR
Ciudad Tratamiento Rango_edad        
Bogotá sr.         18-29        46986
       sra.        18-29        42716
       sr.         30-39        31544
Tunja  sra.        -17          26232
Bogotá sra.        30-39        22022

## Dewey

In [16]:
dewey_count_sr = merge_df.groupby(['Ciudad', 'Tratamiento', 'Rango_edad', 'DEWEY']).CODBAR.count()

In [17]:
len(dewey_count_sr)

39915

In [18]:
sorted_dewey_count_sr = dewey_count_sr.sort_values(ascending=False)

In [19]:
pd.DataFrame(sorted_dewey_count_sr)

CODBAR
Ciudad    Tratamiento Rango_edad DEWEY        
Tunja     sra.        -17        863      6719
          sr.         -17        863      3960
Popayán   sra.        -17        863      3061
Tunja     sra.        -17        813      2700
                                 843      2626
Bogotá    sra.        18-29      863      2593
Tunja     sra.        -17        823      2279
Bogotá    sr.         18-29      863      2081
Popayán   sr.         -17        863      1996
Tunja     sr.         -17        813      1850
                                 843      1800
          sra.        -17        833      1734
Pasto     sra.        -17        863      1473
Tunja     sr.         -17        823      1397
Bogotá    sr.         18-29      813      1319
                      30-39      863      1286
Pereira   sra.        -17        863      1234
Pasto     sr.         -17        863      1224
Tunja     sra.        -17        808      1148
Popayán   sra.        -17        843      1114
Bogotá    sra.        30-39      863      1088
Ipiales   sra.        -17        863      1069
Tunja     sr.         -17        833      1055
Bogotá    sra.        18-29      813      1026
Popayán   sra.        -17        813      1025
Bogotá    sr.         18-29      320      1015
Popayán   sra.        -17        823       982
Bogotá    sra.        18-29      370       972
                                 301       894
          sr.         18-29      301       884
...                                        ...
Florencia sra.        -17        932         1
                                 930         1
Popayán   sr.         30-39      520         1
                                 536         1
                                 541         1
Florencia sra.        -17        915         1
Popayán   sr.         30-39      567         1
Florencia sra.        -17        912         1
Popayán   sr.         30-39      573         1
Florencia sra.        -17        901         1
Popayán   sr.         30-39      589         1
                                 595         1
                                 597         1
Florencia sra.        -17        864         1
                                 794         1
Popayán   sr.         30-39      609         1
                                 625         1
Florencia sra.        -17        859         1
                                 851         1
Popayán   sr.         30-39      632         1
Florencia sra.        -17        842         1
                                 840         1
Popayán   sr.         30-39      634         1
                                 635         1
                                 636         1
Florencia sra.        -17        821         1
Popayán   sr.         30-39      659         1
Florencia sra.        -17        797         1
Popayán   sr.         30-39      665         1
Manizales sr.         +65        180         1

[39915 rows x 1 columns]

In [20]:
filtered_dewey_count_sr = dewey_count_sr[lambda x: x >= THRESHOLD_DEWEY]

In [21]:
len(filtered_dewey_count_sr)

1788

In [22]:
sorted_filtered_dewey_count_sr = filtered_dewey_count_sr.sort_values(ascending=False)

In [23]:
pd.DataFrame(sorted_filtered_dewey_count_sr.head())

CODBAR
Ciudad  Tratamiento Rango_edad DEWEY        
Tunja   sra.        -17        863      6719
        sr.         -17        863      3960
Popayán sra.        -17        863      3061
Tunja   sra.        -17        813      2700
                               843      2626

## Count

### Global

In [24]:
%%time

ciudades = merge_df.Ciudad.unique()
tratamientos = merge_df.Tratamiento.unique()
rangos_edades = merge_df.Rango_edad.unique()

indexes = []
for ciudad in ciudades:
    for tratamiento in tratamientos:
        for rango_edad in rangos_edades:
            index = (ciudad, tratamiento, rango_edad)
            if index in filtered_group_count_sr and index in filtered_dewey_count_sr:
                sr = dewey_count_sr[index]
                sorted_sr = sr.sort_values(ascending=False).head(TOP_GROUP)
                for dewey in sorted_sr.index:
                    indexes.append(index + (dewey,))

CPU times: user 332 ms, sys: 0 ns, total: 332 ms
Wall time: 331 ms


In [25]:
selected_dewey_count_sr = dewey_count_sr[indexes]

In [26]:
len(selected_dewey_count_sr)

1520

In [27]:
selected_dewey_count_sr.head()

Ciudad   Tratamiento  Rango_edad  DEWEY
Armenia  sr.          -17         843      425
                                  863      365
                                  813      281
                                  823      237
                                  833      204
Name: CODBAR, dtype: int64

### Local

In [28]:
from IPython.display import display

In [29]:
pd.set_option('display.max_rows', 120)

def process_group_count(x):
    index = x.name[:2]
    count = group_count_sr[(ciudad, ) + index]
    return count

ciudades = selected_dewey_count_sr.index.levels[0]

for ciudad in ciudades:
    sr = selected_dewey_count_sr[ciudad]
    df = pd.DataFrame(sr)
    df['Cuenta_grupo'] = df.apply(process_group_count, axis=1)
    df = df.rename(columns={'CODBAR': 'Cuenta_dewey'})
    print(ciudad)
    display(df)

pd.reset_option('display.max_rows')

Armenia


Cuenta_dewey  \
Tratamiento Rango_edad DEWEY                                          
sr.         -17        843                                      425   
                       863                                      365   
                       813                                      281   
                       823                                      237   
                       833                                      204   
                       028                                      109   
                       741                                      106   
                       808                                       95   
                       372                                       85   
                       853                                       62   
sra.        40-49      863                                       54   
                       Videograbaciones en sala/pres.            34   
                       813                                       28   
                       843                                       24   
                       Registro sonoro en sala/pres.             23   
                       823                                       23   
                       808                                       22   
                       741                                       19   
                       028                                       15   
                       869                                       12   
            -17        843                                      263   
                       863                                      234   
                       813                                      232   
                       823                                      183   
                       833                                      180   
                       741                                      106   
                       028                                      102   
                       808                                       96   
                       372                                       79   
                       895                                       48   
            50-64      Registro sonoro en sala/pres.             57   
                       863                                       47   
                       371                                       38   
                       370                                       33   
                       153                                       26   
                       813                                       20   
                       372                                       19   
                       Videograbaciones en sala/pres.            16   
                       843                                       15   
                       823                                       14   
            +65        813                                      123   
                       863                                       74   
                       823                                       52   
                       861                                       11   
                       891                                       10   
                       833                                        9   
                       839                                        5   
                       808                                        5   
                       860                                        4   
                       894                                        4   

                                                       Cuenta_grupo  
Tratamiento Rango_edad DEWEY                                         
sr.         -17        843                                     2793  
                       863                                     2793  
                       813                                     2793  
            

Barranquilla


Cuenta_dewey  \
Tratamiento Rango_edad DEWEY                                          
sr.         40-49      Videograbaciones en sala/pres.           179   
                       863                                       47   
                       813                                       34   
                       658                                       33   
                       613                                       18   
                       823                                       16   
                       741                                       13   
                       808                                       10   
                       843                                       10   
                       153                                        9   
            50-64      Registro sonoro en sala/pres.             52   
                       364                                       21   
                       863                                       21   
                       320                                       17   
                       323                                       16   
                       306                                       12   
                       467                                       11   
                       986                                       11   
                       401                                       11   
                       658                                        9   
            18-29      863                                       99   
                       621                                       67   
                       301                                       46   
                       813                                       31   
                       629                                       21   
                       808                                       17   
                       823                                       16   
                       891                                       16   
                       333                                       16   
                       986                                       16   
sra.        18-29      813                                       64   
                       863                                       59   
                       823                                       50   
                       301                                       27   
                       347                                       24   
                       843                                       21   
                       303                                       17   
                       861                                       17   
                       986                                       12   
                       364                                       11   

                                                       Cuenta_grupo  
Tratamiento Rango_edad DEWEY                                         
sr.         40-49      Videograbaciones en sala/pres.           643  
                       863                                      643  
                       813                                      643  
                       658                                      643  
                       613                                      643  
                       823                                      643  
                       741                                      643  
                       808                                      643  
                       843                                      643  
                       153                                      643  
            50-64      Registro sonoro en sala/pres.            491  
                       364                                      491  
                       863                                      491  
                      

Bogotá


...

Cuenta_dewey  \
Tratamiento Rango_edad DEWEY                                          
sr.         40-49      Videograbaciones en sala/pres.           135   
                       863                                      110   
                       621                                       76   
                       813                                       60   
                       986                                       50   
                       301                                       40   
                       330                                       39   
                       333                                       38   
                       370                                       36   
                       823                                       32   
            -17        863                                     3960   
                       813                                     1850   
                       843                                     1800   
                       823                                     1397   
                       833                                     1055   
                       808                                      773   
                       895                                      496   
                       372                                      432   
                       028                                      348   
                       869                                      319   
            30-39      Videograbaciones en sala/pres.           285   
                       863                                      154   
                       843                                       67   
                       813                                       60   
                       823                                       46   
                       330                                       42   
                       658                                       40   
                       338                                       40   
                       Registro sonoro en sala/pres.             38   
                       986                                       36   
            50-64      863                                      296   
                       Videograbaciones en sala/pres.           178   
                       843                                      108   
                       823                                      101   
                       813                                       89   
                       861                                       83   
                       986                                       68   
                       923                                       39   
                       833                                       37   
                       347                                       37   
            +65        863                                       56   
                       986                                       14   
                       861                                        9   
                       621                                        6   
                       813                                        5   
                       Registro sonoro en sala/pres.              4   
                       808                                        4   
                       271                                        4   
                       823                                        4   
                       339                                        4   
            18-29      Videograbaciones en sala/pres.           284   
                       863                                      228   
                       347                                      181   
                       813                                      173   
                       342                                      152   
        

Valledupar


Cuenta_dewey  \
Tratamiento Rango_edad DEWEY                                          
sr.         -17        863                                      105   
                       813                                       96   
                       823                                       56   
                       843                                       47   
                       028                                       25   
                       861                                       21   
                       833                                       20   
                       523                                       17   
                       372                                       15   
                       839                                       12   
            18-29      347                                       87   
                       863                                       66   
                       813                                       64   
                       340                                       61   
                       861                                       54   
                       823                                       47   
                       641                                       42   
                       342                                       41   
                       346                                       38   
                       344                                       36   
sra.        -17        813                                      216   
                       863                                      201   
                       823                                      121   
                       843                                       49   
                       833                                       42   
                       891                                       37   
                       808                                       33   
                       028                                       28   
                       574                                       18   
                       340                                       17   
            30-39      863                                       70   
                       813                                       35   
                       Videograbaciones en sala/pres.            26   
                       823                                       26   
                       808                                       26   
                       330                                       18   
                       843                                       15   
                       895                                       15   
                       861                                       15   
                       028                                       13   
            18-29      863                                       84   
                       813                                       69   
                       331                                       47   
                       347                                       44   
                       823                                       35   
                       342                                       34   
                       341                                       33   
                       150                                       31   
                       861                                       31   
                       343                                       30   

                                                       Cuenta_grupo  
Tratamiento Rango_edad DEWEY                                         
sr.         -17        863                                      682  
                       813                                      682  
                       823                                      682  
            

Villavicencio


Cuenta_dewey  \
Tratamiento Rango_edad DEWEY                                          
sr.         50-64      Registro sonoro en sala/pres.            135   
                       Videograbaciones en sala/pres.            28   
                       863                                        6   
                       Videograbaciones No pres./cons             4   
                       Registro sonoro I. en sala/pre             4   
                       070                                        3   
                       Videograbación I. en sala/pres             2   
                       861                                        2   
                       741                                        1   
                       303                                        1   

                                                       Cuenta_grupo  
Tratamiento Rango_edad DEWEY                                         
sr.         50-64      Registro sonoro en sala/pres.            198  
                       Videograbaciones en sala/pres.           198  
                       863                                      198  
                       Videograbaciones No pres./cons           198  
                       Registro sonoro I. en sala/pre           198  
                       070                                      198  
                       Videograbación I. en sala/pres           198  
                       861                                      198  
                       741                                      198  
                       303                                      198

In [30]:
dewey_dict = {}
with open('dewey2.txt') as f:
    for line in f:
        chunks = line.strip().split()
        dewey = chunks[0]
        topic = ' '.join(chunks[1:])
        dewey_dict[dewey] = topic

In [31]:
def dewey_report(main_dict, sub_dict, cat_dict):
    dewey_code = Counter(main_dict).most_common(1)[0][0]
    dewey_category = dewey_dict[dewey_code]
    print('Top main category:', dewey_code, '-', dewey_category)
    dewey_code = Counter(sub_dict).most_common(1)[0][0]
    dewey_category = dewey_dict[dewey_code]
    print('Top sub category:', dewey_code, '-', dewey_category)
    dewey_code = Counter(cat_dict).most_common(1)[0][0]
    dewey_category = dewey_dict[dewey_code]
    print('Top category:', dewey_code, '-', dewey_category)
    
    for k1 in main_dict:
        print(k1, dewey_dict[k1], ':', main_dict[k1])
        for k2 in sub_dict:
            if k2[:1].startswith(k1[:1]):
                print('    ', k2, dewey_dict[k2], ':', sub_dict[k2])
                for k3 in cat_dict:
                    if k3[:2].startswith(k2[:2]):
                        print('         ', k3, '-', dewey_dict[k3], ':', cat_dict[k3])

In [32]:
ciudades = selected_dewey_count_sr.index.levels[0]

for ciudad in ciudades:
    sr = selected_dewey_count_sr[ciudad]
    for i, subsr in sr.groupby(level=['Tratamiento', 'Rango_edad']):
        main_dict = defaultdict(int)
        sub_dict = defaultdict(int)
        cat_dict = defaultdict(int)
        for j, x in subsr.iteritems():
            try:
                n = int(j[2])
                cat_dict[j[2]] += x
                sub_dict[j[2][:2] + '-'] += x
                main_dict[j[2][:1] + '--'] += x
            except:
                pass
        
        print((ciudad, ) + i)
        dewey_report(main_dict, sub_dict, cat_dict)
        print()
    print()

('Armenia', 'sr.', '-17')
Top main category: 8-- - Literature
Top sub category: 84- - French & related literatures
Top category: 843 - French fiction
7-- Arts & recreation : 106
     74- Graphic arts & decorative arts : 106
          741 - Drawing & drawings : 106
3-- Social sciences : 85
     37- Education : 85
          372 - Primary education (elementary education) : 85
8-- Literature : 1669
     80- Literature, rhetoric & criticism : 95
          808 - Rhetoric & collections of literary texts from more than two literatures : 95
     82- English & Old English literatures : 237
          823 - English fiction : 237
     81- American literature in English : 281
          813 - American fiction in English : 281
     85- Italian, Romanian, & related literatures : 62
          853 - Italian fiction : 62
     83- German & related literatures : 204
          833 - German fiction : 204
     84- French & related literatures : 425
          843 - French fiction : 425
     86- Spanish, Portugu